<a id='0'></a>
# Order Brushing


Tugas:
Mencari Shopid dan userid yang melakukaan brushing. Dengan kriteria dari brushing yaitu userid melakukan pembelian di toko yang sama dalam kurun waktu 1 jam sebanyak 3 kali atau lebih.

Step:
1. [Import library](#1)
2. [Membaca data](#2)
3. [Membuat data frame dengan semua shopid yang unik dengan nilai userid awal 0](#3)
4. [Mencari brushing pada toko dengan 3 transaksi](#4)
5. [Mencari brushing pada toko dengan transaksi lebih dari 3](#5)
6. [Mengeksport hasilnya menjadi csv](#6)
7. [Mengecek akurasi](#7)

**Akurasi = 96,71%**

**Runtime = 2 - 4 menit**

## 1.Import library <a class="anchor" id="1"></a>
Library yang dibutuhkan:
1. pandas untuk mengolah dataframe dan csv
2. datetime untuk mengolah data dengan format waktu
3. statistics untuk mencari modus 
4. collection untuk mencari modus jika terdapat 2 modus (dapat menggunakan mode dari statistics pada python3.8)

In [291]:
import pandas as pd
from datetime import datetime
from statistics import mode
from collections import Counter

## 2. Membaca data <a class="anchor" id="2"></a>
1. Pertama kita rubah kolom event_time menjadi format waktu
2. Untuk memudahkan pengerjaan data diurutkan berdasarkan shopid dan event_time

In [292]:
data = pd.read_csv('order_brush_order.csv')

In [293]:
data['event_time'] = data['event_time'].astype('datetime64[ns]')

In [294]:
data = data.sort_values(['shopid', 'event_time'])

In [295]:
data.head()

,orderid,shopid,userid,event_time
59,31086409141107,10009,196962305,2019-12-27 03:06:50
76187,31144571933461,10051,2854032,2019-12-27 19:16:11
9055,31254979546679,10051,48600461,2019-12-29 01:56:19
188513,31195675919209,10061,168750452,2019-12-28 09:27:55
174620,31205132327893,10061,194819216,2019-12-28 12:05:32


## 3. Membuat data frame untuk jawaban dengan nilai awal semua userid = 0 <a class="anchor" id="3"></a>
Dengan terlebih dahulu kita mengelompokkan data berdasarkan shopid-nya

In [296]:
shops = data.groupby(by='shopid').count()

In [297]:
shops.head()

,orderid,userid,event_time
shopid,,,
10009,1,1,1
10051,2,2,2
10061,4,4,4
10084,55,55,55
10100,42,42,42


In [381]:
csv = pd.DataFrame({'shopid' : shops.index, 'userid' : [0 for i in range(len(shops))]}).set_index('shopid')

In [382]:
csv.head()

,userid
shopid,
10009,0
10051,0
10061,0
10084,0
10100,0


## 4. Mencari brushing pada toko dengan 3  transaksi <a class="anchor" id="4"></a>
Step:
1. Mencari shopsid dengan orderid  = 3
2. Mencari brushing dengan kriteria: 1 shopid hanya terdapat 1 userid dan transaksi dilakukan kurang dari 1 jam
3. Mengubah nilai pada dataframe jawaban dengan nilai userid yang telah ditemukan

In [383]:
susp1 =  shops[shops['orderid'] == 3].index

In [384]:
#Mencari brush user untuk shop dengan 3 transaksi
susps = []  #shopid yang melakukan brushing
suspu = []  #userid yang melakukan brushing
for i in susp1:
    dt = data[data['shopid']==i]
    time0 = dt.iloc[0,3]
    time1 = dt.iloc[-1,3]
    if len(set(dt['userid'])) == 1 and ((time1-time0).total_seconds() <= 3600):
        susps.append(dt.iloc[0,1])
        suspu.append(dt.iloc[0,2])

In [385]:
#Menambahkan userid yang telah terdeteksi brushing ke dataframe jawaban
csv['userid'][susps] = suspu

In [386]:
csv.loc[susps]

,userid
shopid,
3124091,214568881
50682734,214365114
129113839,215408773
153444897,169948888
153672697,214778616
155016230,188431260
157939195,69563623
161196859,180862522
163351089,214778616


## 5. Mencari brushing pada toko dengan transaksi lebih dari 3 <a class="anchor" id="5"></a>
Step:
1. Mencari shopid dengan orderid > 3
2. Membuat dataframe baru dari shopid yang didapatkan
3. Mencari brushing dengan fungsi getuser
4. Menggabungkan hasil yang didapat dengan dataframe csv 

In [304]:
susp2 = shops[shops['orderid'] > 3].index

In [305]:
data2 = data[data['shopid'].isin(susp2)]

In [356]:
data2.head()

,orderid,shopid,userid,event_time
188513,31195675919209,10061,168750452,2019-12-28 09:27:55
174620,31205132327893,10061,194819216,2019-12-28 12:05:32
200439,31412115824794,10061,130633421,2019-12-30 21:35:15
145426,31431527100615,10061,62464559,2019-12-31 02:58:48
167859,31075686185309,10084,4401933,2019-12-27 00:08:06


**Cara kerja fungsi**
1. Fungsi bekerja dengan dataframe dari 1 shopid saja
2. Mencari transaksi yang terjadi dalam 1 jam. Jumlah transaksasi akan bertambah jika jeda transaksi kurang dari 1 jam dan akan terus berulang hingga jeda lebih dari 1 jam
3. Jika jumlah transaksi dibagi jumlah userid lebih dari sama dengan 3 maka userid dengan transaksi terbanyak dikategorin brushing
4. Dilihat dari jumlah transaksinya jika terdapat userid yang melakukan transaksi lebih dari sama dengan  3 maka dikategorikan brushing


In [360]:
#Fungsi untuk mencari userid yang brushing dengan input dataframe dengan 1 shopid
def getuser(test):
    sub = []
    for i in range((len(test))):
        usr = 1
        usrn = [test.iloc[i,2]]
        time0 = test.iloc[i,3]
        for j in range(i+1, len(test)):
            time1 = test.iloc[j,3]
            if (time1 - time0).total_seconds() <= 3600:
                usr+=1
                usrn.append(test.iloc[j,2])
            else:
                break
        if len(usrn) > 0 and usr/len(set(usrn)) >= 3:
            try:
                sub.append(mode(usrn)) 
            except:
                usrn = Counter(usrn)
                sub.extend([usrn.most_common()[i][0] for i in range(2)])
        else:
            for k in set(usrn):
                if usrn.count(k) >= 3:
                    sub.append(k)
    if len(sub) == 0 or len(set(sub)) > 2: 
        return 0
    else:
        sub = list(set(sub))
        sub.sort()
        return(sub)

**Cara kerja fungsi**
1. Input : List shopid 
2. Membuat dataframe baru dari 1 shopid
3. Mengaplikasikan fungsi getuser untuk setiap data frame yang baru
4. Output : List shopid dan userid yang terdeteksi brushing

In [376]:
#Mengaplikasikan fungsi getuser ke dataframe soal
def brush(up):
    ids = [] #shopid yang brushing
    idu = [] #userid yang brushing
    for i in (up):
        test = data2[data2['shopid'] == i].reset_index(drop=True)
        ans = getuser(test)
        if ans:
            ids.append(i)
            idu.append(ans)
    return [ids, idu]

In [402]:
%%time
brushs = brush(susp2)

CPU times: user 3min 4s, sys: 51.9 ms, total: 3min 4s
Wall time: 3min 4s


In [403]:
len(brushs[0])

371

In [398]:
csv['userid'][brushs[0]] = brushs[1]

In [399]:
#Membuat userid menjadi string dan menambahkan &
csv['userid'][brushs[0]] = csv['userid'][brushs[0]].apply(lambda x: '&'.join([str(i) for i in x]))

In [400]:
csv[csv['userid'] != 0]

,userid
shopid,
10159,214988798
10287,198097381&214226569
10402,77819
10536,672345
29583,214495220&215275495
...,...
203531250,114282846
204225676,198662175
208696908,214111334


## 6. Mengekspor hasilnya menjadi CSV <a class="anchor" id="6"></a>

In [216]:
csv.to_csv('uji.csv', index=False)

## 7. Mengecek akurasi <a class="anchor" id="7"></a>
Dengan membandingkan dengan hasil dengan akurasi 1. Perhitungannya jika userid benar tetapi useridnya 0 maka bernilai 0.005 jika hasilnya benar dan useridnya bukan 0 maka nilainya 1 selain itu nilainya 0.

In [189]:
cek = pd.read_csv('submission.csv').sort_values('shopid').set_index('shopid')

In [373]:
def cek_acc(csv):
    acc = 0
    for i in range(len(csv)):
        if str(csv.iloc[i,0]) == cek.iloc[i,0] and cek.iloc[i,0] !='0':
            acc+=1
        elif str(csv.iloc[i,0]) == cek.iloc[i,0] and cek.iloc[i,0] =='0':
            acc+=0.005
        else:
            acc+=0
    return(acc/407.275)

In [401]:
cek_acc(csv)

0.9671720581914554